# Algoritmia
## Práctica Obligatoria 1
### Curso 2023 - 2024
###### Métodos Voraces
---
 

#### Autores:
* Jimena Arnaiz González
* Iván Estépar Rebollo

---
Resuelva la siguiente práctica.

Importe las librerías que desees
**Recuerda**: 
* Solamente puedes utilizar bibliotecas nativas (https://docs.python.org/es/3.8/library/index.html)
* Las funciones que importes no son "gratis", cada una tendrá una complejidad temporal y espacial que se tendrá que tener en cuenta.

In [1]:
#testeable
# Imports


In [2]:
#testeable
class Video:
    """
    Clase Video. 
    Representa una serie o película.
    """    
    
    def __init__(self, name, size):
        """Crea un objeto de clase Video

        Parameters
        ----------
        name : str
            Nombre de la serie/película
        size : number
            Tamaño en memoria de la serie/película
        """
        self.name = name
        self.size = size
        self.dicc_users = {} #diccionario country-users (almacena la cantidad de espectadores por país)
        
    
    def __hash__(self):
        """Genera el valor hash identificativo del vídeo

        Returns
        -------
        int
            Valor hash
        """        
        return hash(self.name)
    
    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """        
        return f"Vídeo {self.name} de tamaño {self.size}" #devuelve el nombre y tamaño del video
    
    def __repr__(self):
        """Genera una cadena descriptiva del objeto dentro de colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """  
        return self.__str__()
       
    def set_users(self, country, users):
        """Dado un pais y un número de usuarios
           almacena para este vídeo la cantidad de espectadores que tiene.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película
        users : int
            Número de espectadores
        """   
        #Si el país ya está en el diccionario, actualiza la cantidad de usuarios
        if country in self.dicc_users:
            self.dicc_users[country] += users
        #si no, se agrega una nueva entrada
        else:
            self.dicc_users[country] = users
            
    
    def get_users(self, country):
        """Dado un país, obtiene el número de usuarios.

        Parameters
        ----------
        country : str
            País desde donde se ve la serie/película

        Returns
        -------
        int
            Número de espectadores para el país `country`
        """
        return self.dicc_users.get(country, 0) # Devuelve 0 si no hay usuarios registrados para el país dado

In [3]:
#testeable
class ServidorCache:
    """
    Clase del servidor caché donde se almacenan parte de series/películas.
    """
    
    def __init__(self, identifier, country, capacity):
        """Instancia un Servidor de Caché

        Parameters
        ----------
        identifier : int
            Valor que identifica un servidor.
        country : str
            País donde está el servidor.
        capacity : int
            Cantidad de memoria de almacenamiento disponible.
        """
        self.identifier = identifier
        self.country = country
        self.capacity = capacity

        self.cantVid = {} #diccionario que almacena video-cantidad de los videos alcamenados
        
    def __hash__(self):
        """Genera el valor hash identificativo del servidor

        Returns
        -------
        int
            Valor hash
        """     
        return hash(self.identifier)
    
    def __str__(self):
        """Genera una cadena descriptiva del objeto

        Returns
        -------
        str
            Cadena descriptiva
        """      
        return f"Servidor {self.identifier} de tamaño {self.capacity} situado en {self.country}"
        
    def __repr__(self):
        """Genera una cadena descriptiva del objeto en colecciones

        Returns
        -------
        str
            Cadena descriptiva
        """      
        return f"Servidor {self.identifier} de tamaño {self.capacity} situado en {self.country}"
            
    def rellena(self, videos):
        """Dada una colección de videos,
           seleccionar de cada uno cuanta cantidad (entre 0 y 1)
           se almacena en el servidor.
           Se ha de optimizar para que el tiempo de emisión  #param a calcular con un algoritmo creo
           sea el máximo posible.

        Parameters
        ----------
        videos : collection
            Colección de videos que se quieren almacenar en el servidor.
        """
        #Ordena los videos por cantidad de espectadores en orden descendente
        vidsOrdenados = sorted(videos, key = lambda v: v.get_users(self.country), reverse=True)
        #Peso acumulado de los videos almacenados en el servidor
        wAcum = 0 
    
        #Algoritmo de la Mochila
        #Itera sobre los videos ordenados          
        for v in vidsOrdenados:
            #Si todavía hay espacio en el servidor
            if wAcum < self.capacity:
                #Si el peso acumulado más el tamaño del video no excede la capacidad del servidor
                if wAcum + v.size <= self.capacity: 
                    #Metemos el video entero (1)
                    self.cantVid[v] = 1
                    wAcum = wAcum + v.size
                #Si no, se almacena una fracción del video para no exceder la capacidad del servidor
                else:
                    self.cantVid[v] = (self.capacity - wAcum) / v.size 
                    wAcum = self.capacity
            else:
                break
            
            
    def disponible(self, video):
        """Obtiene la cantidad de vídeo disponible en el servidor.

        Parameters
        ----------
        video : Video object
            Vídeo del cual se quiere saber la disponibilidad

        Returns
        -------
        float
            Cantidad del vídeo disponible          
        """ 
        return self.cantVid.get(video)
    
    def almacenados(self):
        """Material almacenado en el servidor

        Returns
        -------
        set
            Conjunto de tuplas (video, cantidad) de los videos ALMACENADOS en el servidor.          
        """ 
        setVidAlmac = set() #set para almacenar las tuplas de videos y su cantidad almacenada en el servidor
        
        #Itera sobre los videos y su cantidad almacenada en el servidor y los agrega al set
        for vid, cant in self.cantVid.items():
            tuplaVidAlmac = (vid, cant) 
            setVidAlmac.add(tuplaVidAlmac)
            
        return setVidAlmac
    
    
    def tiempo_emision(self):
        """A partir de los datos almacenados
           devolver el tiempo de emisión
           siguiendo la fórmula: 
           \sum_{i}^{v} \text{espectadores}_i*\text{tamaño}_i*\text{porcionAlmacenada}_i

        Returns
        -------
        number
            Tiempo de emision disponible
        """        
        tEmision = 0
        
        #Itera sobre los videos y la cantidad de video almacenados en el servidor y calcula el tiempo de emisión
        for vid, cant in self.almacenados():
            tEmision += vid.size * cant * vid.get_users(self.country)
            
        return tEmision
   

In [4]:
#testeable
class ServidorMaestro:
    """
    Servidor central que gestiona las conexiones entre servidores cache
    """
    
    def __init__(self, servidores, distancias):
        """Instancia el servidor central

        Parameters
        ----------
        servidores : Iterable
            Conjunto de servidores cache disponibles
        distancias : dict{ServidorCache: dict{ServidorCache: int}}   p.e: {A:{B:10,C:5}}
            Grafo de distancias en milisegundos entre servidores.
        """        
        self.servidores = set(servidores)
        self.distancias = distancias

    
    def get_grafo(self):
        """Devuelve el grafo de distancias recibido

        Returns
        -------
        dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.
        """ 
        return self.distancias

    def get_grafo_simplificado(self):
        """Devuelve el grafo de distancias simplificado

        Returns
        -------
        dict{ServidorCache: dict{ServidorCache: int}}
            Grafo de distancias en milisegundos entre servidores.
        """        
        return self.simplifica_grafo()     
          
    def simplifica_grafo(self):
        """A partir del grafo de distancias
           hacer una simplificación de la estrucutra
           de datos para ahorrar espacio y tiempo.
        """
        '''#Solo necesitamos un camino para representar la distancia entre dos servidores,
        #por lo que tomamos la distancia mínima entre ellos.
        
        grafo_simplificado = {} #diccionario vacío para el grafo simplificado

        #Itera sobre cada servidor en el grafo de distancias
        for servidor in self.distancias:
            #Crea un diccionario vacío para el servidor en el grafo simplificado
            grafo_simplificado[servidor] = {}
            
            #Itera sobre cada vecino del servidor en el grafo de distancias
            for vecino in self.distancias[servidor]:
                #Calcula la distancia min entre el servidor y su vecino
                #si vecino no tiene una entrada directa al servidor servidor, devuelve un diccionario vacío,
                #y si no hay una entrada directa desde vecino a servidor, devuelve infinito
                min_distancia = min(self.distancias[servidor][vecino], self.distancias.get(vecino, {}).get(servidor, float('inf')))
                #Asigna la distancia mínima al grafo simplificado
                grafo_simplificado[servidor][vecino] = min_distancia
                #Agrega la conexión inversa al grafo simplificado
                grafo_simplificado.setdefault(vecino, {})[servidor] = min_distancia

        return grafo_simplificado'''
        
         # Inicializar el árbol de expansión mínimo
        arbol_expansion_minimo = {}
        
        # Conjunto de servidores visitados
        visitados = set()
        
        # Inicializar el algoritmo con un servidor arbitrario
        primer_servidor = next(iter(self.servidores))
        visitados.add(primer_servidor)
        
        # Utilizar el algoritmo de Prim para construir el árbol de expansión mínimo
        while len(visitados) < len(self.servidores):
            min_distancia = float('inf')
            min_servidor_origen = None
            min_servidor_destino = None
            
            # Buscar la arista con la menor distancia hacia un servidor no visitado
            for servidor_origen in visitados:
                for servidor_destino, distancia in self.distancias[servidor_origen].items():
                    if servidor_destino not in visitados and distancia < min_distancia:
                        min_distancia = distancia
                        min_servidor_origen = servidor_origen
                        min_servidor_destino = servidor_destino
            
            # Agregar la arista al árbol de expansión mínimo
            arbol_expansion_minimo.setdefault(min_servidor_origen, {})[min_servidor_destino] = min_distancia
            arbol_expansion_minimo.setdefault(min_servidor_destino, {})[min_servidor_origen] = min_distancia
            visitados.add(min_servidor_destino)
        
        return arbol_expansion_minimo
           
    
    def mas_cercano(self, servidor):
        """Reporta el servidor más cercano al dado por parámetro

        Parameters
        ----------
        servidor : ServidorCache

        Returns
        -------
        ServidorCache
            Servidor más cercano
        """ 
    
        # Eliminamos el propio servidor de la lista
        no_visitados = self.servidores.copy()
        no_visitados.remove(servidor)
    
        # Encuentra el servidor más cercano en función de la distancia mínima
        servidor_mas_cercano = min(no_visitados, key=lambda x: self.distancias[servidor][x])
    
        return servidor_mas_cercano
        

### Caso de ejemplo

In [5]:
import unittest
import json

def carga_dataset(data):
    with open(data) as f:    
        test_datasets = json.load(f)

    videos = list()
    for v in test_datasets["videos"]:
        v_obj = Video(v["name"], v["size"])
        for c, u in v["users"].items():
            v_obj.set_users(c, u)
        videos.append(v_obj)
            
    servers = dict()
    for s in test_datasets["servers"]:
        servers[s["country"]] = ServidorCache(s["identifier"], s["country"], s["size"])

    
    pings = test_datasets["pings"]
    p_ = dict()
    for s in servers.values():
        p_[s] = dict()
        for p in pings[s.country]:
            p_[s][servers[p]] = pings[s.country][p]
    maestro = ServidorMaestro(servers.values(), p_)
    
    return videos, servers, maestro

In [6]:
class TestBasico(unittest.TestCase):
    
    def test_carga_simple(self):
        
        v, s, m = carga_dataset("toy.json")

        spain = s["Spain"]
        spain.rellena(v)
        self.assertEqual(spain.tiempo_emision(), 578000)
        almacenados = spain.almacenados()
        self.assertIn((v[3], 0.5), almacenados)
        
        m.simplifica_grafo()
        self.assertEqual(m.mas_cercano(s["Spain"]), s["France"])
        m.simplifica_grafo()
        self.assertEqual(m.mas_cercano(s["Spain"]), s["France"])
        self.assertEqual(m.mas_cercano(s["France"]), s["Spain"])

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.011s

OK


##### **Tests**

Para probar que tu solución pasa los tests. Utilice el comando:

```bash
$ python tests-py3<version de python> <mi notebook>
```

Los tests necesitan de las librerías `networkx` y `nbformat`

```bash
$ pip install networkx nbformat
```

###### Explicación de los tests
* `test_ejemplo`: Es el mismo que el caso de ejemplo.
* `test_ej1_emision_correcta`: Comprueba que el tiempo de emisión del servidor caché es correcto.
* `test_ej1_sin_espacio`: Comprueba que ante un servidor sin espacio, el tiempo de emisión es 0.
* `test_ej1_espacio_infinito`: Comprueba que ante un servidor con espacio infinito, el tiempo de emisión es el máximo.
* `test_ej1_pais_no_existe`: Comprueba que ante pais que no tiene servidor cache, el tiempo de emisión es 0.
* `test_ej2_estructura_datos_mas_simple`: Comprueba que la estructura de datos que se utiliza para almacenar la red de servidores es más simple que la original.
* `test_ej2_red_servidores_consistente`: Comprueba que la red de servidores es constitente con el mapa original, es decir, no hay conexiones nuevas y los costes son los mismos.
* `test_ej2_sistema_conexo`: Comprueba que la red de servidores cache es conexa.

---

### **NOTA**
<br>
Al ejecutar este archivo en la máquina virtual, nos aparece el siguiente fail en el TestBasico:

FAIL: test_ejemplo (__main__.TestBasico)

Traceback (most recent call last):
  File "<frozen PO1-Voraces_test>", line 91, in test_ejemplo
AssertionError: (Vídeo Moonfall de tamaño 400, 0.5) not found in {(Vídeo Encanto de tamaño 500, 1), (Vídeo Belfast de tamaño 300, 1), (Vídeo Moonfall de tamaño 400, 0.5)} 

Sin embargo, al ejecutarlo en el propio jupyter, lo pasa correctamente.


### **Informe**
Contesta a las siguientes preguntas.

#### **Complejidad**

1. Método `ServidorCache.rellena`
    * **Complejidad temporal**: La complejidad temporal de esta función depende principalmente del tamaño de la colección de videos y de cómo se implementa la ordenación de los videos. Dado que el tiempo de ejecución más significativo es el de la ordenación de los videos, la complejidad temporal total de esta función es O(n log n), donde 'n' es el número de videos en la colección.  
    
<br>

2. Método `ServidorMaestro.simplifica_grafo`
    * **Complejidad temporal**: Dado que el peor de los casos es cuando todos los servidores están completamente conectados entre sí, tanto directa como inversamente, y cada servidor tiene n vecinos, podemos concluir que la complejidad temporal total del algoritmo es de O(n^2). Esto es porque las operaciones de bucle anidado dominan la complejidad.


#### **Servidores cache.**

* ¿La solución es óptima (maximiza siempre el tiempo de emisión) o es aproximada (encuentra un máximo local)?

Es aproximada. La solución proporcionada por el método rellena no garantiza una solución óptima en términos de maximizar siempre el tiempo de emisión. La estrategia de ordenar los videos por la cantidad de usuarios en un país específico y luego asignar porciones de cada video según la capacidad disponible del servidor es una aproximación heurística que intenta maximizar el tiempo de emisión, pero no garantiza un máximo global. --MAL

---bien: Es optima porque se llena la mochila

* ¿Qué ocurriría si solo se admitiese almacenar vídeos completos en cada servidor?

Si solo se permitiera almacenar videos completos en cada servidor, la situación cambiaría. En este caso, la capacidad de almacenamiento del servidor determinaría cuántos y qué videos se pueden almacenar. La estrategia de selección sería más simple: almacenar los videos con mayor número de usuarios en el país del servidor hasta que la capacidad se agote. Esta estrategia garantizaría que se maximice el tiempo de emisión para los videos almacenados, ya que no habría porciones de video fraccionadas.


#### **Red de servidores cache**

* ¿La solución es óptima (la red es lo más simple posible) o es aproximada (encuentra un mínimo local)?

La solución proporcionada es aproximada, ya que utiliza una estrategia heurística para intentar maximizar el tiempo de emisión, pero no garantiza una solución óptima en términos de maximizar siempre el tiempo de emisión. El algoritmo utilizado es una aproximación que ordena los videos por la cantidad de usuarios en un país específico y luego asigna porciones de cada video según la capacidad disponible del servidor. Sin embargo, esta estrategia no garantiza un máximo global y podría encontrar solo un máximo local en algunos casos. --MAL

--bien
 La solución proporcionada por este algoritmo es óptima y no aproximada. No busca un máximo local, sino que encuentra la solución que minimiza la distancia total entre los servidores cache, asegurando así una transmisión más eficiente de los datos.

* ¿Cómo afecta el número de conexiones entre servidores a la complejidad temporal del algoritmo empleado?

El número de conexiones entre servidores puede afectar significativamente la complejidad temporal del algoritmo empleado. Si hay un gran número de conexiones entre servidores, la complejidad del algoritmo podría aumentar debido a la necesidad de coordinar y optimizar el almacenamiento de videos entre múltiples servidores. 